<a href="https://colab.research.google.com/github/ryotaD/project_colab/blob/master/4fans_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id = '1b_FhRyOmX5oZz4Oj10Mg1bafgqYjhFBg'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_data.zip')

In [0]:
id = '1SFlWFPr4pnNV6gz-xTXRX-eqO7wGK1dP'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test_data.zip')

In [0]:
id = '13-sSZFEfsum4RVNhdh9PeMQ5p6ezluQj'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_labels.csv')

In [0]:
!unzip train_data.zip
!unzip test_data.zip

Archive:  train_data.zip
replace train_data/0.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: train_data/0.png        
  inflating: train_data/1.png        
  inflating: train_data/10.png       
  inflating: train_data/100.png      
  inflating: train_data/1000.png     
  inflating: train_data/10000.png    
  inflating: train_data/10001.png    
  inflating: train_data/10002.png    
  inflating: train_data/10003.png    
  inflating: train_data/10004.png    
  inflating: train_data/10005.png    
  inflating: train_data/10006.png    
  inflating: train_data/10007.png    
  inflating: train_data/10008.png    
  inflating: train_data/10009.png    
  inflating: train_data/1001.png     
  inflating: train_data/10010.png    
  inflating: train_data/10011.png    
  inflating: train_data/10012.png    
  inflating: train_data/10013.png    
  inflating: train_data/10014.png    
  inflating: train_data/10015.png    
  inflating: train_data/10016.png    
  inflating: train_data/10017.png  

In [0]:
import pandas as pd
train_labels = pd.read_csv('train_labels.csv', index_col=False, dtype=object)

In [0]:
!rm train_data.zip
!rm test_data.zip


**Basic preprocessing**

In [0]:
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [0]:
from natsort import natsorted
train_image_folders = natsorted(glob.glob("train_data/*"))
test_image_folders = natsorted(glob.glob("test_data/*"))

In [0]:
import pandas as pd
y_train = []
for i in train_labels["Label"].values:
    for j in list(i):
        y_train.append(j)
y_train = np.array(y_train)

In [0]:
from PIL import Image, ImageFilter
def load_images(image_files):
    images = []
    for f in image_files:
        im = np.array(Image.open(f))
        images.append(im)
    return np.array(images)

In [0]:
train_images = load_images(train_image_folders)
X_train = np.array(train_images).astype('float32') / 255

In [0]:
del train_images

In [0]:
import keras.utils
y_train = []
for i in train_labels["Label"].values:
    for j in list(i):
        y_train.append(j)
y_train = np.array(y_train)
y_train = keras.utils.to_categorical(y_train, num_classes=10)

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential, Model
from keras.layers import Input, Flatten, Dense

In [0]:
img_width, img_height = 256, 256
input_tensor = Input(shape=(img_width, img_height, 3))
ResNet50 = ResNet50(include_top=False, weights='imagenet',input_tensor=input_tensor)

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [0]:
top_model = Sequential()
top_model.add(Flatten(input_shape=ResNet50.output_shape[1:]))
top_model.add(Dense(nb_classes, activation='softmax'))

In [0]:
model = Model(input=ResNet50.input, output=top_model(ResNet50.output))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  """Entry point for launching an IPython kernel.


In [0]:
from keras import optimizers
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.Adam(),metrics=['accuracy'])

In [0]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [0]:
history = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=20,
    validation_split=0.1,
)

Train on 17051 samples, validate on 1895 samples
Epoch 1/20
17051/17051 [==============================] - 166s 10ms/step - loss: 3.1361 - acc: 0.1197 - val_loss: 6.0955 - val_acc: 0.1235
Epoch 2/20
17051/17051 [==============================] - 152s 9ms/step - loss: 2.7535 - acc: 0.1262 - val_loss: 2.3862 - val_acc: 0.1266
Epoch 3/20
17051/17051 [==============================] - 152s 9ms/step - loss: 2.7235 - acc: 0.1267 - val_loss: 2.3949 - val_acc: 0.1277
Epoch 4/20
17051/17051 [==============================] - 152s 9ms/step - loss: 2.6946 - acc: 0.1275 - val_loss: 2.4049 - val_acc: 0.1282
Epoch 5/20
17051/17051 [==============================] - 152s 9ms/step - loss: 2.6668 - acc: 0.1406 - val_loss: 2.4119 - val_acc: 0.1282
Epoch 6/20
17051/17051 [==============================] - 152s 9ms/step - loss: 2.5769 - acc: 0.1941 - val_loss: 2.3062 - val_acc: 0.1916
Epoch 7/20
17051/17051 [==============================] - 152s 9ms/step - loss: 1.9197 - acc: 0.5060 - val_loss: 1.2011 - 

In [0]:
test_images = load_images(test_image_folders)
X_test = np.array(test_images).astype('float32') / 255
del test_images